##### Copyright 2018 The TensorFlow Authors.



In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Image Captioning with Attention

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/beta/tutorials/text/image_captioning">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/r2.0rc/site/en/r2/tutorials/text/image_captioning.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/r2.0rc/site/en/r2/tutorials/text/image_captioning.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/r2/tutorials/text/image_captioning.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Given an image like the example below, our goal is to generate a caption such as "a surfer riding on a wave".

![Man Surfing](https://tensorflow.org/images/surf.jpg)

*[Image Source](https://commons.wikimedia.org/wiki/Surfing#/media/File:Surfing_in_Hawaii.jpg); License: Public Domain*

To accomplish this, you'll use an attention-based model, which enables us to see what parts of the image the model focuses on as it generates a caption.

![Prediction](https://tensorflow.org/images/imcap_prediction.png)

The model architecture is similar to [Show, Attend and Tell: Neural Image Caption Generation with Visual Attention](https://arxiv.org/abs/1502.03044).

This notebook is an end-to-end example. When you run the notebook, it downloads the [MS-COCO](http://cocodataset.org/#home) dataset, preprocesses and caches a subset of images using Inception V3, trains an encoder-decoder model, and generates captions on new images using the trained model.

In this example, you will train a model on a relatively small amount of data—the first 30,000 captions  for about 20,000 images (because there are multiple captions per image in the dataset).

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import pandas as pd
import urllib.request as request
import progressbar
import time
import threading
from shutil import copyfile
import cv2
import zipfile
import glob

import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle

## Extract images and annotations

In [0]:
zip_images_file_path = "/content/drive/Shared drives/AISight/COCO/train2014.zip"
zip_annotations_file_path = "/content/drive/Shared drives/AISight/COCO/annotations_trainval2014.zip"
extracted_file_path = "/content/drive/Shared drives/AISight/COCO/Dataset"

In [0]:
with zipfile.ZipFile(zip_images_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_file_path)

In [0]:
with zipfile.ZipFile(zip_annotations_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_file_path)

## Get images and related captions
Each image has 5 captions. 

In [0]:
annotation_file_path = "/content/drive/Shared drives/AISight/COCO/Dataset/annotations/captions_train2014.json"
images_dir_path = "/content/drive/Shared drives/AISight/COCO/Dataset/train2014"
image_name_format = "COCO_train2014_000000$.jpg"
npy_files = "/content/drive/Shared drives/AISight/COCO/npy_files"

In [0]:
image_name_length = 12

## Remove .npy files

In [0]:
total_files = [path for path in os.listdir(images_dir_path)]

In [0]:
len(total_files)

In [0]:
num_thread = 20
per_thread_photos = int(len(total_files)/num_thread)
per_thread_photos

In [0]:
def remove_npy(start, end):
  for path in total_files[start:end]:
    if len(path.split(".")) == 3:
      os.remove(os.path.join(images_dir_path, path))

In [0]:
t = []
for i in range(num_thread):
    arg1 = i * per_thread_photos
    if i == (num_thread-1):
        arg2 = len(total_files)
    else:
        arg2 = i * per_thread_photos + per_thread_photos
    # print(arg1, arg2)
    t.append(threading.Thread(target=remove_npy, args=(arg1, arg2, )))
    t[i].start()
      
for i in range(num_thread):
    t[i].join()

## Find total .npy files

In [0]:
total_files = [path for path in os.listdir(images_dir_path)]

In [0]:
total_files = [f for dirpath, dirnames, filenames in os.walk(images_dir_path) for f in filenames ]

In [0]:
total_files = []
for dirpath, dirnames, filenames in os.walk(images_dir_path):
  print(dirpath)
  print(dirnames)
  for f in filenames:
    print(f)

In [0]:
jpg_files = glob(images_dir_path+"/*.jpg")

In [0]:
 len(jpg_files)

In [0]:
npy_files = glob(images_dir_path+"/*.jpg.npy")

In [0]:
len(npy_files), npy_files[0]

In [0]:
total_files = npy_files

In [0]:
number_of_npy = [item for item in total_files if len(item.split(".")) == 3]

In [0]:
len(number_of_npy), number_of_npy[0]

In [0]:
images_to_consider = [item.split(".")[0] + "." + item.split(".")[1] for item in npy_files]

In [0]:
len(images_to_consider), images_to_consider[0]

In [0]:
with open(annotation_file_path) as f:
    anno = json.load(f)

In [0]:
anno_captions_list = anno['annotations']

In [0]:
images = []
captions = []
for item in anno_captions_list:
  images.append(item['image_id'])
  captions.append(item['caption'])

In [0]:
len(images), len(captions)

In [0]:
def replace_image_path(x):
  length_image_id = len(str(x))
  name = ""
  if length_image_id == 1:
    name = "00000" + str(x)  
  elif length_image_id == 2:
    name = "0000" + str(x)  
  elif length_image_id == 3:
    name = "000" + str(x)  
  elif length_image_id == 4:
    name = "00" + str(x)  
  elif length_image_id == 5:
    name = "0" + str(x) 
  else:
    name = str(x) 
  image_name = image_name_format.replace("$", name)
  # print(image_name)
  image_path = os.path.join(images_dir_path, image_name)
  # print(image_path)
  return image_path

In [0]:
temp = list(map(replace_image_path,images))

In [0]:
temp[0]

In [0]:
images = []
new_captions = []
for i, item in enumerate(temp):
  # print(item)
  if item in images_to_consider:
    # print("if")
    images.append(item)
    new_captions.append(captions[i])

In [0]:
len(images), len(new_captions)

In [0]:
def add_path(x):
  return os.path.join(images_dir_path, x)

In [0]:
temp = list(map(add_path,images))npy_files = "/content/drive/Shared drives/AISight/COCO/npy_files"

In [0]:
del images
del captions

In [0]:
# images = temp
captions = new_captions

In [0]:
len(images), len(captions), images[0]

In [0]:
df = pd.DataFrame(zip(images, captions), columns=["image","caption"])

In [0]:
df.head()npy_files = "/content/drive/Shared drives/AISight/COCO/npy_files"

In [0]:
len(list(set(images)))

## Get all images and captions

In [0]:
images_name = [name for name in os.listdir(images_dir_path)]

In [0]:
len(images_name), images_name[0]

In [0]:
with open(annotation_file_path) as f:
    anno = json.load(f)

In [0]:
anno_captions_list = anno['annotations']

In [0]:
len(anno_captions_list)

In [0]:
images = []
captions = []
for item in anno_captions_list:
  images.append(item['image_id'])
  captions.append(item['caption'])

In [0]:
len(images), len(captions)

In [0]:
def replace_image_path(x):
  length_image_id = len(str(x))
  name = ""
  if length_image_id == 1:
    name = "00000" + str(x)  
  elif length_image_id == 2:
    name = "0000" + str(x)  
  elif length_image_id == 3:
    name = "000" + str(x)  
  elif length_image_id == 4:
    name = "00" + str(x)  
  elif length_image_id == 5:
    name = "0" + str(x) 
  else:
    name = str(x) 
  image_name = image_name_format.replace("$", name)
  # print(image_name)
  image_path = os.path.join(images_dir_path, image_name)
  # print(image_path)
  return image_path

In [0]:
temp = list(map(replace_image_path,images))

In [0]:
len(temp), temp[0]

In [0]:
images = temp

In [0]:
df = pd.DataFrame(zip(images, captions), columns=["image","caption"])

In [0]:
df.head()

## Download Google Conceptual dataset


In [0]:
train_file = "/content/drive/Shared drives/AISight/Google Dataset/Train_GCC-training.tsv"
gadgets_file = "/content/drive/Shared drives/AISight/Google Dataset/gadgets.csv"
photos_dir = "/content/drive/Shared drives/AISight/Google Dataset/gadgets_photos_pny"
error_file = "/content/drive/Shared drives/AISight/Google Dataset/Error Files"
print (os.path.exists(train_file))

In [0]:
df = pd.read_csv(gadgets_file, names=['Captions', 'Urls'])

In [0]:
df.shape

In [0]:
links = df['Urls']

In [0]:
df.iloc[9]

In [0]:
def download_images(start, end):
  print (start, end)
  error = []
  bar = progressbar.ProgressBar(max_value=(end-start))
  for i in range(start, end):
    # print (i)
    try:
      request.urlopen(links[i], timeout=8)
      request.urlretrieve(links[i],  photos_dir + "/" + str(i) + ".jpg")
    except:
      error.append(i)
    bar.update(i-start)
  return error

In [0]:
def thread_fun(start, end):
  error = download_images(start, end)
  with open(os.path.join(error_file, "error_" + str(start) + "_" + str(end) + ".txt"), 'w+') as file:
    for e in error:
      file.write(str(e) + ",") 

In [0]:
for i in range(6,10):
  thread_fun(i*1000,i*1000+1000)

In [0]:
print(len([name for name in os.listdir(photos_dir)]))

In [0]:
print(len(error) , "not downloaded")

## Check and remove file if it is not image
Try to open the file using PIL library's Image module and if fails delete it - **Multithreading** 

In [0]:
num_thread = 30
photos_path = [patnpy_files = "/content/drive/Shared drives/AISight/COCO/npy_files"h for path in os.listdir(photos_dir)]
per_thread_photos = int(len(photos_path)/num_thread)

In [0]:
def check_photos(start,end):
  bar = progressbar.ProgressBar(max_value=(end-start))
  for i, path in zip(range(start,end), photos_path[start:end]):
    try:
      Image.open(os.path.join(photos_dir, path))
    except:
      os.remove(os.path.join(photos_dir, path))
    bar.update(i-start)

In [0]:
t = []
for i in range(num_thread):
    arg1 = i * per_thread_photos
    if i == (num_thread-1):
        arg2 = len(photos_path)
    else:
        arg2 = i * per_thread_photos + per_thread_photos
    # print(arg1, arg2)
    t.append(threading.Thread(target=check_photos, args=(arg1, arg2, )))
    t[i].start()
      
for i in range(num_thread):
    t[i].join()

## Get captions and photos path

In [0]:
photos_index = [int(i.split(".")[0]) for i in os.listdir(photos_dir)]

In [0]:
photos_index = list(set(photos_index))

In [0]:
photos_index[:5], len(photos_index)

In [0]:
processed_df = df.iloc[photos_index]

In [0]:
processed_df_index = processed_df.index.tolist()

In [0]:
photos_path = [os.path.join(photos_dir,str(i) + ".jpg") for i in processed_df_index]

In [0]:
photos_path[:5]

In [0]:
captions = processed_df['Captions'].tolist()

In [0]:
final_df = pd.DataFrame({"Captions":captions, "Photo_paths":photos_path})

In [0]:
print(final_df['Captions'].iloc[1], final_df['Photo_paths'].iloc[1])

In [0]:
len(captions)

In [0]:
# Store captions and image names in vectors
all_captions = []
all_img_name_vector = photos_path

for c in captions:
    caption = '<start> ' + c + ' <end>'
    all_captions.append(caption)

# Shuffle captions and image_names together
# Set a random state
train_captions, img_name_vector = shuffle(all_captions,
                                          all_img_name_vector,
                                          random_state=1)

In [0]:
len(train_captions),len(img_name_vector)

In [0]:
img_name_vector[:2]

## Make captions and image path list


In [0]:
# Store captions and image names in vectors
all_captions = []
all_img_name_vector = []

for index, row in df.iterrows():
    c = row['caption']
    im = row['image']
    caption = '<start> ' + c + ' <end>'
    all_captions.append(caption)
    all_img_name_vector.append(im)

# Shuffle captions and image_names together
# Set a random state
train_captions, img_name_vector = shuffle(all_captions,
                                          all_img_name_vector,
                                          random_state=1)

In [0]:
len(train_captions),len(img_name_vector)

## Preprocess the images using InceptionV3
Next, you will use InceptionV3 (which is pretrained on Imagenet) to classify each image. You will extract features from the last convolutional layer.

First, you will convert the images into InceptionV3's expected format by:
* Resizing the image to 299px by 299px
* [Preprocess the images](https://cloud.google.com/tpu/docs/inception-v3-advanced#preprocessing_stage) using the [preprocess_input](https://www.tensorflow.org/api_docs/python/tf/keras/applications/inception_v3/preprocess_input) method to normalize the image so that it contains pixels in the range of -1 to 1, which matches the format of the images used to train InceptionV3.

In [0]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    # path = npy_files + "/" + image_path.decode("utf-8").split("/")[-1]
    return img, image_path

## Initialize InceptionV3 and load the pretrained Imagenet weights

Now you'll create a tf.keras model where the output layer is the last convolutional layer in the InceptionV3 architecture. The shape of the output of this layer is ```8x8x2048```. You use the last convolutional layer because you are using attention in this example. You don't perform this initialization during training because it could become a bottleneck.

* You forward each image through the network and store the resulting vector in a dictionary (image_name --> feature_vector).
* After all the images are passed through the network, you pickle the dictionary and save it to disk.




In [0]:
image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output
image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

## Caching the features extracted from InceptionV3

You will pre-process each image with InceptionV3 and cache the output to disk. Caching the output in RAM would be faster but also memory intensive, requiring 8 \* 8 \* 2048 floats per image. At the time of writing, this exceeds the memory limitations of Colab (currently 12GB of memory).

Performance could be improved with a more sophisticated caching strategy (for example, by sharding the images to reduce random access disk I/O), but that would require more code.

The caching will take about 10 minutes to run in Colab with a GPU. If you'd like to see a progress bar, you can: 

1. install [tqdm](https://github.com/tqdm/tqdm):

    `!pip install tqdm`

2. Import tqdm:

    `from tqdm import tqdm`

3. Change the following line:

    `for img, path in image_dataset:`

    to:

    `for img, path in tqdm(image_dataset):`


## Photos numpy array
Create a numpy array of all training images - **Multithreading**


In [0]:
num_thread = 8
per_thread_photos = int(len(img_name_vector)/num_thread)

In [0]:
image = np.zeros((len(img_name_vector),299,299,3))
img_path = [""] * len(img_name_vector) 

In [0]:
def generate_photos_array(m,n):
  for i, path in zip(range(m,n), img_name_vector[m:n]):
    # if i == 5:
    #   breakimage_model
    a, b = load_image(path)
    image[i] = a
    img_path[i] = b

In [0]:
t = []
for i in range(num_thread):
    arg1 = i * per_thread_photos
    if i == (num_thread-1):
        arg2 = len(img_name_vector)
    else:
        arg2 = i * p
for index, row in df.iterrows():
    c = row['caption']er_thread_photos + per_thread_photos
    print(arg1, arg2)
    t.append(threading.Thread(target=generate_photos_array, args=(arg1, arg2, )))
    t[i].start()
        
for i in range(num_thread):image_model
    t[i].join()

In [0]:
image.shape

In [0]:
# Get unique images
encode_train = sorted(set(img_name_vector))

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)


for img, path in zip(image, img_path):
  img = img.reshape(1,299,299,3)
  # print(img.shape)
  batch_features = image_features_extract_model(img)
  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    print(path_of_feature)
    np.save(path_of_feature, bf.numpy())

In [0]:
encode_train = sorted(set(img_name_vector))

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

# bar = progressbar.ProgressBar(max_value=(end-start))

for img, path in image_dataset:
  if len(img) == 1:
    img_name_vector = filter(lambda x : x == path , img_name_vector)
  else:
    batch_features = image_features_extract_model(img)
    batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

    for bf, p in zip(batch_features, path):
      path_of_feature = p.numpy().decode("utf-8")
      np.save(path_of_feature, bf.numpy())

In [0]:
num_thread = 10
per_thread_photos = int(len(list(set(img_name_vector)))/num_thread)
per_thread_photos

In [0]:
def save_photos_as_numpy(start, end):
  
  # Get unique images
  encode_train = sorted(set(img_name_vector[start:end]))

  # Feel free to change batch_size according to your system configuration
  image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
  image_dataset = image_dataset.map(
    load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

  bar = progressbar.ProgressBar(max_value=(end-start))

  for i, (img, path) in zip(range(start, end), image_dataset):
    batch_features = image_features_extract_model(img)
    batch_features = tf.reshape(batch_features,
                                (batch_features.shape[0], -1, batch_features.shape[3]))

    for bf, p in zip(batch_features, path):
      path_of_feature = p.numpy().decode("utf-8")
      np.save(path_of_feature, bf.numpy(ssBar(max_value=(end-start))
))
    bar.update(i-start)

In [0]:
t = []
for i in range(num_thread):
    arg1 = i * per_thread_photos
    if i == (num_thread-1):
        arg2 = len(list(set(img_name_vector)))
    else:
        arg2 = i * per_thread_photos + per_thread_photos
    # print(arg1, arg2)
    t.append(threading.Thread(target=save_photos_as_numpy, args=(arg1, arg2, )))
    t[i].start()
        
for i in range(num_thread):
    t[i].join()

In [0]:
len([path for path in os.listdir(images_dir_path)])

In [0]:
print(len([path for path in os.listdir("/content/drive/Shared drives/AISight/Google Dataset/gadgets")]))

In [0]:
gadgets_dir = [path for path in os.listdir(photos_dir)]

In [0]:
pny = [path for path in gadgets_dir if len(path.split(".")) == 3]

In [0]:
len(pny)

In [0]:
photos = [path for path in gadgets_dir if len(path.split(".")) == 2]

In [0]:
len(photos)

In [0]:
google_dataset = "/content/drive/Shared drives/AISight/Google Dataset"

In [0]:
google_dataset = "/content/drive/Shared drives/AISight/Google Dataset"
if 'gadgets_photos_pny' not in os.listdir(google_dataset):
  os.mkdir(os.gadgets_photos_pnypath.join(google_dataset, "gadgets_photos_pny"))

In [0]:
for p in pny:
  copyfile(photos_dir + "/" + p.split(".")[0] + ".jpg" , google_dataset + "/" "gadgets_photos_pny" + "/" + p.split(".")[0] + ".jpg")
  copyfile(photos_dir + "/" + p, google_dataset + "/" "gadgets_photos_pny" + "/" + p )

In [0]:
# train_captions, img_name_vector
photos_from_pny = [item for item in os.listdir(os.path.join(google_dataset, "gadgets_photos_pny")) if len(item.split(".")) == 2]
photos_from_pny = list(set(photos_from_pny))
print(len(photos_from_pny))
final_captions = []
final_name_vector = []
for p in phot/conimage_modeltent/drive/Shared drives/AISight/Google Dataset/gadgetsos_from_pny:
  index = img_name_vector.index(photos_dir + "/" + p)
  final_captions.append(train_captions[index])
  final_name_vector.append(img_name_vector[index])

In [0]:
final_captions[:5], final_name_vector[:5]

In [0]:
Image.open(final_name_vector[1])

In [0]:
del train_captions
del img_name_vector

In [0]:
train_captions = final_captionsimage_model
img_name_vector = final_name_vector 

In [0]:
temp = []
for i in range(len(img_name_vector)):
  temp.append(google_dataset + "/" + "gadgets_photos_pny/" + img_name_vector[i].split("/")[-1])

In [0]:
img_name_vector[:2], temp[:2], len(img_name_vector), len(temp)

In [0]:
del img_name_vector
img_name_vector = temp

In [0]:
del temp

In [0]:
temp = [path.split(".")[0] for path in os.listdir(google_dataset + "/" + "gadgets_photos_pny")]
# temp = list(set(temp))
print(len(temp))
# for path in temp:
#   if temp.count(path) == 2:
#     continue
#   else:
#     os.remove(google_dataset + "/" + "gadgets_photos_pny/" + path + ".jpg")

## Make .npy files

In [0]:
num_thread = 7
per_thread_photos = int(len(list(set(img_name_vector)))/num_thread)
print(per_thread_photos)

In [0]:
def save_photos_as_numpy(start, end):
  
  # Get unique images
  encode_train = sorted(set(img_name_vector[start:end]))

  # Feel free to change batch_size according to your system configuration
  image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
  image_dataset = image_dataset.map(
    load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

  bar = progressbar.ProgressBar(max_value=(end-start))

  count = 0
  for i, (img, path) in zip(range(start, end), image_dataset):
    if os.path.exists(path):
      count += 1
      continue
    batch_features = image_features_extract_model(img)
    batch_features = tf.reshape(batch_features,
                                (batch_features.shape[0], -1, batch_features.shape[3]))

    for bf, p in zip(batch_features, path):
      path_of_feature = p.numpy().decode("utf-8")
      np.save(path_of_feature, bf.numpy())
    if count % 25 == 0:
      print(count)
    bar.update(i-start)

In [0]:
t = []
for i in range(num_thread):
    arg1 = i * per_thread_photos
    if i == (num_thread-1):
        arg2 = len(list(set(img_name_vector)))
    else:
        arg2 = i * per_thread_photos + per_thread_photos
    # print(arg1, arg2)
    t.append(threading.Thread(target=save_photos_as_numpy, args=(arg1, arg2, )))
    t[i].start()
        
for i in range(num_thread):
    t[i].join()

In [0]:
encode_train = sorted(set(img_name_vector))

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

# bar = progressbar.ProgressBar(max_value=(end-start))

for img, path in image_dataset:
  batch_features = image_features_extract_model(img)
  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())
  # bar.update(i-start)

## Preprocess and tokenize the captions

* First, you'll tokenize the captions (for example, by splitting on spaces). This gives us a  vocabulary of all of the unique words in the data (for example, "surfing", "football", and so on).
* Next, you'll limit the vocabulary size to the top 5,000 words (to save memory). You'll replace all other words with the token "UNK" (unknown).
* You then create word-to-index and index-to-word mappings.
* Finally, you pad all sequences to be the same length as the longest one.

In [0]:
# Find the maximum length of any caption in our dataset
def calc_max_length(tensor):
    return max(len(t) for t in tensor)

In [0]:
# Choose thebar = progressbar.ProgressBar(max_value=(end-start)) top 5000 words from the vocabulary
top_k = 5000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(train_captions)
train_seqs = tokenizer.texts_to_sequences(train_captions)

In [0]:
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

In [0]:
# Create the tokenized vectors
train_seqs = tokenizer.texts_to_sequences(train_captions)

In [0]:
# Pad each vector to the max_length of the captions
# If you do not provide a max_length value, pad_sequences calculates it automatically
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

In [0]:
# Calculates the max_length, which is used to store the attention weights
max_length = calc_max_length(train_seqs)

## Split the data into training and testing

In [0]:
# Create training and validation sets using an 80-20 split
img_name_train, img_name_val, cap_train, cap_val = train_test_split(img_name_vector,
                                                                    cap_vector,
                                                                    test_size=0.2,
                                                                    random_state=0)

In [0]:
len(img_name_train), len(cap_train), len(img_name_val), len(cap_val)

In [0]:
img_name_train[0]

## Create a tf.data dataset for training



 Our images and captions are ready! Next, let's create a tf.data dataset to use for training our model.

In [0]:
# Feel free to change these parameters according to your system's configuration

BATCH_SIZE = 64
BUFFER_SIZE = 1000

embedding_dim = 256
units = 512
vocab_size = len(tokenizer.word_index) + 1
num_steps = len(img_name_train) // BATCH_SIZE
# Shape of the vector extracted from InceptionV3 is (64, 2048)
# These two variables represent that vector shape
features_shape = 2048
attention_features_shape = 64

In [0]:
# Load the numpy files
def map_func(img_name, cap):
  img_tensor = np.load(img_name.decode('utf-8')+'.npy')
  return img_tensor, cap

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))

# Use map to load the numpy files in parallel
dataset = dataset.map(lambda item1, item2: tf.numpy_function(
          map_func, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Shuffle and batch
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

## Model

Fun fact: the decoder below is identical to the one in the example for [Neural Machine Translation with Attention](../sequences/nmt_with_attention.ipynb).

The model architecture is inspired by the [Show, Attend and Tell](https://arxiv.org/pdf/1502.03044.pdf) paper.

* In this example, you extract the features from the lower convolutional layer of InceptionV3 giving us a vector of shape (8, 8, 2048).
* You squash that to a shape of (64, 2048).
* This vector is then passed through the CNN Encoder (which consists of a single Fully connected layer).
* The RNN (here GRU) attends over the image to predict the next word.

In [0]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, features, hidden):
    # features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)

    # hidden shape == (batch_size, hidden_size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
    hidden_with_time_axis = tf.expand_dims(hidden, 1)

    # score shape == (batch_size, 64, hidden_size)
    score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

    # attention_weights shape == (batch_size, 64, 1)
    # you get 1 at the last axis because you are applying score to self.V
    attention_weights = tf.nn.softmax(self.V(score), axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [0]:
class CNN_Encoder(tf.keras.Model):
    # Since you have already extracted the features and dumped it using pickle
    # This encoder passes those features through a Fully connected layer
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x

In [0]:
class RNN_Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units, vocab_size):
    super(RNN_Decoder, self).__init__()
    self.units = units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc1 = tf.keras.layers.Dense(self.units)
    self.fc2 = tf.keras.layers.Dense(vocab_size)

    self.attention = BahdanauAttention(self.units)

  def call(self, x, features, hidden):
    # defining attention as a separate model
    context_vector, attention_weights = self.attention(features, hidden)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # shape == (batch_size, max_length, hidden_size)
    x = self.fc1(output)

    # x shape == (batch_size * max_length, hidden_size)
    x = tf.reshape(x, (-1, x.shape[2]))

    # output shape == (batch_size * max_length, vocab)
    x = self.fc2(x)

    return x, state, attention_weights

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))

In [0]:
encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoint

In [0]:
checkpoint_path = "./checkpoints/train"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [0]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
  start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])

## Training

* You extract the features stored in the respective `.npy` files and then pass those features through the encoder.
* The encoder output, hidden state(initialized to 0) and the decoder input (which is the start token) is passed to the decoder.
* The decoder returns the predictions and the decoder hidden state.
* The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
* Use teacher forcing to decide the next input to the decoder.
* Teacher forcing is the technique where the target word is passed as the next input to the decoder.
* The final step is to calculate the gradients and apply it to the optimizer and backpropagate.


In [0]:
# adding this in a separate cell because if you run the training cell
# many times, the loss_plot array will be reset
loss_plot = []

In [0]:
@tf.function
def train_step(img_tensor, target):
  loss = 0

  # initializing the hidden state for each batch
  # because the captions are not related from image to image
  hidden = decoder.reset_state(batch_size=target.shape[0])

  dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

  with tf.GradientTape() as tape:
      features = encoder(img_tensor)

      for i in range(1, target.shape[1]):
          # passing the features through the decoder
          predictions, hidden, _ = decoder(dec_input, features, hidden)

          loss += loss_function(target[:, i], predictions)

          # using teacher forcing
          dec_input = tf.expand_dims(target[:, i], 1)

  total_loss = (loss / int(target.shape[1]))

  trainable_variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, trainable_variables)

  optimizer.apply_gradients(zip(gradients, trainable_variables))
  return loss, total_loss

In [0]:
EPOCHS = 1

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset):
        try:
          batch_loss, t_loss = train_step(img_tensor, target)
          total_loss += t_loss
        except:
          continue

        if batch % 100 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(
              epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
    # storing the epoch end loss value to plot later
    loss_plot.append(total_loss / num_steps)

    # if epoch % 5 == 0:
    #   ckpt_manager.save()
    ckpt_manager.save()

    print ('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                         total_loss/num_steps))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [0]:
plt.plot(loss_plot)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Plot')
plt.show()

## Caption!

* The evaluate function is similar to the training loop, except you don't use teacher forcing here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the end token.
* And store the attention weights for every time step.

In [0]:
def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

In [0]:
def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(10, 10))

    len_result = len(result)
    for l in range(len_result):
        temp_att = np.resize(attention_plot[l], (8, 8))
        ax = fig.add_subplot(len_result//2, len_result//2, l+1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()

In [0]:
# captions on the validation set
rid = np.random.randint(0, len(img_name_val))
image = img_name_val[rid]
real_caption = ' '.join([tokenizer.index_word[i] for i in cap_val[rid] if i not in [0]])
result, attention_plot = evaluate(image)

print ('Real Caption:', real_caption)
print ('Prediction Caption:', ' '.join(result))
plot_attention(image, result, attention_plot)
# opening the image
Image.open(img_name_val[rid])

## Try it on your own images
For fun, below we've provided a method you can use to caption your own images with the model we've just trained. Keep in mind, it was trained on a relatively small amount of data, and your images may be different from the training data (so be prepared for weird results!)


In [0]:
# image_url = 'https://tensorflow.org/images/surf.jpg'
# image_extension = image_url[-4:]
# image_path = tf.keras.utils.get_file('image'+image_extension,
#                                      origin=image_url)
# image_path = os.path.join(photos_dir,"1150.jpg")
# image_path = img_name_vector[13465]
image_path = "/content/IMG_20191104_194850.jpg"
im = Image.open(image_path)
im.thumbnail((512,512), Image.ANTIALIAS)
im.save(image_path, "JPEG")
result, attention_plot = evaluate(image_path)
print ('Prediction Caption:', ' '.join(result))
# plot_attention(image_path, result, attention_plot)
# opening the image
Image.open(image_path)

# Next steps

Congrats! You've just trained an image captioning model with attention. Next, take a look at this example [Neural Machine Translation with Attention](../sequences/nmt_with_attention.ipynb). It uses a similar architecture to translate between Spanish and English sentences. You can also experiment with training the code in this notebook on a different dataset.